# Lab | Transformers

---

### Section structure

1. The open-source ecosystem: increasing accessibility to machine learning (ML) software and hardware
2. Some simple code demonstrations
3. Q&A

## 1. Ease-of-use: Using Transformers in 3 lines of code


**Overview of different tasks that can be automated with ML**
* Key ingredients: (1) a model trained on a specific task; (2) input data (e.g. texts or images); (3) output produced by the model.
* Transformers are currently the most popular type of deep learning algorithm. Most tasks below are solved with Transformers. There might be other types of algorithms coming up in the medium term.



**Install the Transformers library & dependencies**

In [4]:
!pip install transformers  # The Transformers library from Hugging Face
!pip install sentencepiece
!pip install wikipedia
!pip install accelerate
!pip install tf-keras
!pip install torch

# NOTE: you might need to restart you jupyter kernel after installing the libraries

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=e85b9a3374fe805e07226ff35be0cfc064ea0b9fee3d73c4fa9f99f2601a6931
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


**The Hugging Face Pipeline**
* Makes automation of many NLP tasks possible in 3 lines of code
* Detailed documentation is available [here](https://huggingface.co/transformers/main_classes/pipelines.html)

In [3]:
from transformers import pipeline
import pandas as pd
import numpy as np
from pprint import pprint

Note : You might need more libraries or updates to run the cells below, if that is the case, follow the error messages and pip install accordingly. Chat gpt can help you if given the error messages.

### 2.1 Many models tailored to specific tasks


#### 2.1.1 Text classification

Let's select a popular text classification model in the [HF model hub](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads).

Here we chose "cardiffnlp/twitter-roberta-base-irony".

We will classify text into ironic or non ironic.

In [3]:
pipeline_classification = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-irony")  # cardiffnlp/twitter-roberta-base-irony, SamLowe/roberta-base-go_emotions

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


Now that we have the model we can pass it a string and have it give us a classification.

Feel free to experiment with different sentences by changing the contents of the variable text

In [5]:
text = "Well that workshop was totally worth my time..."  # "Well that workshop was totally worth my time..."  "This smells weird, I'm not sure if I should eat this ... Yikes, it tasted like old socks!"
output = pipeline_classification(text, top_k=10)
print(output)

[{'label': 'irony', 'score': 0.9424386620521545}, {'label': 'non_irony', 'score': 0.05756137892603874}]


Let's make the output a little cleaner

In [6]:
# make output a bit cleaner
df_output = pd.DataFrame(output)
print(df_output)

       label     score
0      irony  0.942439
1  non_irony  0.057561


As you can see, in a few lines of code and by leveraging an existing model we can classify text as ironic or non ironic. Now you have one more tool in your machine learning toolbox.

Remember that : 'when you only have a hammer everything is a nail'. But if we want to build a house (perform machine learning the right way), we need to use the right tool for the right job.

#### 2.1.2 Machine Translation

* Open source machine translation (MT) models enable you to translate between many different languages without Google Translate.
* [University of Helsinki](https://huggingface.co/Helsinki-NLP) uploaded models for more than 1000 language pairs to the Hugging Face hub
* [Facebook AI](https://huggingface.co/models?search=facebook+m2m) open-sourced several multi-lingual models
* The [EasyNMT library](https://github.com/UKPLab/EasyNMT), provides an easy wrapper for all these models
* Most machine translation models translate between two languages in one direction (e.g. German to English, but not English to German), some can translate in multiple directions.


In [7]:
# translation pipeline docs: https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TranslationPipeline
pipeline_translate = pipeline("translation", model="facebook/m2m100_418M")

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Device set to use cuda:0


M2M100 is a multilingual encoder-decoder (seq-to-seq) model trained for Many-to-Many multilingual translation.

The model that can directly translate between the 9,900 directions of 100 languages.

Here we specify to translate from German 'de' to English 'en'

In [8]:
text = "Ich bin ein Fisch"
pipeline_translate(text, src_lang="de", tgt_lang="en")

[{'translation_text': 'I am a fish'}]

Let's do the same but with and entire wikipedia page in german.

In [9]:
# download any text from wikipedia, via  https://pypi.org/project/wikipedia/
import wikipedia
wikipedia.set_lang("de")

text = wikipedia.summary("Donald Trump").replace('\n', ' ')[:318]
print(f"Original text:\n{text}\n")

# translate the text from wikipedia
text_translated = pipeline_translate(text, src_lang="de", tgt_lang="en")
print(f"Translated text:\n{text_translated[0]['translation_text']}")


Original text:
Donald John Trump [ˈdɑn.əld dʒɑn tɹɐmp] (* 14. Juni 1946 in New York City) ist ein US-amerikanischer Politiker der Republikanischen Partei, Unternehmer und ehemaliger Showmaster. Von 2017 bis 2021 war er der 45. Präsident der Vereinigten Staaten. Er siegte auch bei der Präsidentschaftswahl im November 2024 und ist so

Translated text:
Donald John Trump [ˈdɑn.əld dʒɑn tɔmp] (born 14 June 1946 in New York City) is an American politician of the Republican Party, entrepreneur and former showmaster. From 2017 to 2021 he was the 45th President of the United States.


#### 2.1.3 Text Summarization

In [ ]:
# docs for summarisation pipeline: https://huggingface.co/transformers/main_classes/pipelines.html#summarizationpipeline
pipeline_summarize = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")  # sshleifer/distilbart-cnn-12-6 , google/pegasus-cnn_dailymail

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# download any long text from wikipedia, via  https://pypi.org/project/wikipedia/
import wikipedia
wikipedia.set_lang("en")

text_long = wikipedia.summary("Donald Trump").replace('\n', ' ')
print(f"Original text:\n{text_long}\n")

# translate the text from wikipedia
text_summarized = pipeline_summarize(text_long, min_length=5, max_length=30)
print(f"Summarized text:\n{text_summarized[0]['summary_text']}")

Original text:
Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. Having won the 2024 presidential election as the nominee of the Republican Party, he is the president-elect and will be inaugurated as the 47th president on January 20, 2025. Trump graduated with a bachelor's degree in economics from the University of Pennsylvania in 1968. He became president of the Trump family real estate business in 1971 and focused on luxury accommodations. After a series of business bankruptcies in the 1990s, he launched several side ventures. From 2004 to 2015, he hosted and co-produced the reality television series The Apprentice.  Trump won the 2016 presidential election as the Republican nominee. His immigration policy included a travel ban targeting several Muslim-majority countries and expanding the U.S.–Mexico border wall; he also briefly implemented a family separation p

#### 2.1.4 Named Entity Recognition

NER is a task that involves identifying and classifying specific entities in text into predefined categories, such as names of people, organizations, locations, dates, and more.

For example, in the sentence "Apple Inc. was founded by Steve Jobs in California," NER would recognize "Apple Inc." as an organization, "Steve Jobs" as a person, and "California" as a location.

In [ ]:
pipeline_ner = pipeline("token-classification", model="dslim/bert-base-NER-uncased", aggregation_strategy="simple")

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import wikipedia
wikipedia.set_lang("en")

text_long = wikipedia.summary("Donald Trump").replace('\n', ' ')

output = pipeline_ner(text_long)

pd.DataFrame(output)

,entity_group,score,word,start,end
0,PER,0.985987,donald john trump,0,17
1,MISC,0.994277,american,45,53
2,LOC,0.994215,united states,141,154
3,MISC,0.849193,republican,238,248
4,ORG,0.523571,party,249,254
5,PER,0.986137,trump,349,354
6,ORG,0.671986,university of pennsylvania,412,438
7,PER,0.787140,trump,475,480
8,MISC,0.940491,the apprentice,719,733
9,PER,0.985755,trump,736,741


### 2.2. Universal models

The models mentioned above are designed to excel at a single specific task on a particular dataset. The key advantage of these models is their high performance and accuracy on that specific task and dataset.

However, in real-world applications, the problems you'll face often require solving slightly different tasks, possibly with varied category definitions or applied to different types of texts.

Universal models can help address this challenge. Although they also focus on one task, the task is general or universal enough that many other tasks can be reformulated into it. Two examples of universal tasks are:

- Natural Language Inference (NLI): A task that can effectively solve a wide range of classification tasks by determining whether a given premise supports, contradicts, or is neutral with respect to a hypothesis.

- Token Generation: An even more universal task that can be applied to solve virtually any text-related task, including translation, summarization, and text completion.

These universal tasks enable the models to be versatile and adaptable to various problems beyond the specific ones they were initially trained on.

#### Zero-shot classification


Zero-shot classification is a technique where a model can categorize data into classes it has never seen before.

Instead of relying on labeled examples for each class, the model understands the relationship between the input and the class descriptions, allowing it to make accurate predictions without needing specific training on those classes.

In [ ]:
pipeline_zeroshot_classification = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0


Here we will give the model a list of classes ('payment issues', 'travel advice', 'bug report') for it to classify our string.

In [ ]:
text = "Customer: I have not received my reimbursement yet. What the hell is going on?"
classes = ['payment issues', 'travel advice', 'bug report']  # "account opening", "customer complaint"

#text = "I do not think the government is trustworthy anymore. We need to mobilize and resist!"
#classes = ["civil disobedience", "praise of the government", "travel advice"]  # "collective action"

output = pipeline_zeroshot_classification(text, classes, multi_label=True)

pd.DataFrame(data=[output["labels"], output["scores"]], index=["class", "probability"]).T


,class,probability
0,payment issues,0.991133
1,bug report,0.076115
2,travel advice,0.018696


## Exercise

Now it is your turn to go to the hugging face library https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads

(you can select on the left menu of the website the type of NLP tasks you want models to perform)

- Find an NLP model that we have not used previously.

- Get some data from wikipedia or elsewhere.

- Perform inference with the model and print the result!

- Comment your code along the way, describe what your model does and what your end goal is from input to output.

Have fun!

In [9]:
# Your code here :

from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification
import wikipedia

# Load the tokenizer and model
tokenizer = XLMRobertaTokenizerFast.from_pretrained('s-nlp/xlmr_formality_classifier')
model = XLMRobertaForSequenceClassification.from_pretrained('s-nlp/xlmr_formality_classifier')

# Map label IDs to formality categories
id2formality = {0: "formal", 1: "informal"}

# Set the Wikipedia language
wikipedia.set_lang("en")

# Function to classify the formality of a Wikipedia article
def classify_formality(article_title):
    try:
        # Fetch the content of the Wikipedia article
        article_content = wikipedia.page(article_title).content

        # Prepare the input text (trim to 512 tokens to match model limits)
        encoding = tokenizer(
            article_content[:512],
            add_special_tokens=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        # Perform inference
        output = model(**encoding)
        logits = output.logits.softmax(dim=1)

        # Convert logits to formality scores
        formality_scores = {id2formality[idx]: score for idx, score in enumerate(logits[0].tolist())}

        # Return the most likely formality classification and its score
        classification = max(formality_scores, key=formality_scores.get)
        confidence = formality_scores[classification]

        return {"classification": classification, "confidence": confidence, "scores": formality_scores}
    except wikipedia.exceptions.DisambiguationError as e:
        return {"error": f"DisambiguationError: {article_title} refers to multiple articles: {e.options}"}
    except wikipedia.exceptions.PageError:
        return {"error": f"PageError: The page '{article_title}' does not exist."}
    except Exception as e:
        return {"error": f"An unexpected error occurred: {e}"}

# Example: Classify a Wikipedia article for formality
article_title = "Elizabeth II"  # Replace with your article title
result = classify_formality(article_title)

if "error" in result:
    print(result["error"])
else:
    print(f"Classification: {result['classification']} (Confidence: {result['confidence']:.2f})")
    print(f"Scores: {result['scores']}")






Classification: formal (Confidence: 1.00)
Scores: {'formal': 0.9986264705657959, 'informal': 0.0013735926477238536}


In [15]:
article_title = "Dublin"  # Replace with your article title
result = classify_formality(article_title)

if "error" in result:
    print(result["error"])
else:
    print(f"Classification: {result['classification']} (Confidence: {result['confidence']:.2f})")
    print(f"Scores: {result['scores']}")

Classification: formal (Confidence: 1.00)
Scores: {'formal': 0.99863201379776, 'informal': 0.0013680103002116084}
